# Running the `ialg` algorithm on the TSPLIB and the HardTSPLIB instances

Here, we want to compute the minimum number of SECs needed to prove optimality for some small famous instances in the TSPLIB [1] and some hard-to-solve instances in the HardTSPLIB [2]. We will use the `ialg` algorithm to compute the minimum number of SECs needed to prove optimality for these instances.

### References
[1] Reinelt, Gerhard. "TSPLIB—A traveling salesman problem library." ORSA journal on computing 3.4 (1991): 376-384.

[2] Vercesi, Eleonora, et al. "On the generation of metric TSP instances with a large integrality gap by branch-and-cut." Mathematical Programming Computation 15.2 (2023): 389-416.


In [18]:
from ialg import ialg, mip
from cover import set_cover_subroutine
from utils import from_tsplib_file_to_graph
import pandas as pd

## TSPLIB instances

These are the instances we were able to conduct the experiment on. Bigger instances were not feasible to run on our machine. If you want to test other instances, just add them to the `tsplib_instances` list at position 0.

In [19]:
tsplib_instances = [("burma14", 14), ("ulysses16", 16), ("gr17", 17), ("ulysses22", 22), ("fri26", 29), ("bayg29", 29), ("bays29", 29), 
                     ("dantzig42", 42), ("swiss42", 42), ("att48", 48), ("gr48", 48), ("hk48", 48), ("eil51", 51), ("berlin52", 52),
                    ("brazil58", 58), ("st70", 70), ("eil76", 76), ("pr76", 76)]

In [20]:
# Store the values in a dictionary
out = {}

Now, we run the `ialg` algorithm on the TSPLIB instances. Unfortunately, this may take a while. If you want to make it faster, just reduce the `max_instance` index

In [21]:
max_instance = len(tsplib_instances)

In [ ]:
for instance_name, n in tsplib_instances:
    # Parse the instance
    G = from_tsplib_file_to_graph("./data/" + instance_name)
    print("******* Instance:", instance_name, "*******")
    (S_family, size_S_family, partitions, c, runtime) = ialg(G, verbose=True)
    
    if size_S_family == -1:
        print("Ran into time limit.")
        continue
        
    # check that k* >= size_S_family
    partitions_list = [ [ list(part) for part in partition ] for npts in partitions for partition in partitions[npts] ]
    smallest_S_family = set_cover_subroutine(partitions_list, verbose=False)
    print("k* =",size_S_family,"for S_family =",smallest_S_family)
    assert len(smallest_S_family) == size_S_family
    
    # check that k* <= size_S_family
    two_factor_cost = mip(G, initial_subtours=smallest_S_family, verbose=False)
    tsp_cost = mip(G, subtour_callbacks=True, verbose=False)
    assert round(two_factor_cost) == round(tsp_cost)
    
    print(" ") # Leave some space

******* Instance: burma14 *******
TSP compute in 0.005486726760864258 seconds. TSP cost = 3323
With smart initialization, we begin with #SECs = 2
They are:
frozenset({2, 3, 4, 5, 6, 11, 12, 13})
frozenset({0, 1, 2, 3, 4, 5, 6, 7, 11, 12, 13})
Found a solution with #SECs: 2
Specifically, they are:
frozenset({2, 3, 4, 5, 6, 11, 12, 13})
frozenset({0, 1, 2, 3, 4, 5, 6, 7, 11, 12, 13})
k* = 2 for S_family = [frozenset({0, 1, 7, 8, 9, 10}), frozenset({8, 9, 10})]
 
******* Instance: ulysses16 *******
TSP compute in 0.010118961334228516 seconds. TSP cost = 6859
With smart initialization, we begin with #SECs = 4
They are:
frozenset({0, 1, 2, 3, 7})
frozenset({4, 5, 6, 8, 9, 10, 14})
frozenset({4, 5, 6, 8, 9, 10, 11, 12, 13, 14})
frozenset({0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15})
Found a solution with #SECs: 4
Specifically, they are:
frozenset({0, 1, 2, 3, 7})
frozenset({4, 5, 6, 8, 9, 10, 14})
frozenset({4, 5, 6, 8, 9, 10, 11, 12, 13, 14})
frozenset({0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,

Now, we print the table as reported in the paper

In [27]:
# Create a dataframe out of the dictionary out
df = pd.DataFrame(out.items(), columns=["instance", "S_min"])
# Print the dataframe

,instance,S_min
0,burma14,2
1,ulysses16,4
2,gr17,5
3,ulysses22,5
4,fri26,4
5,bayg29,4
6,bays29,5
7,swiss42,3
8,dantzig42,4


## HardTSPLIB instances

HardTSPLIB is made of instances generated both at random and starting from instances of the TSPLIB. Unfortunately, we were not able to run the algorithm on the instances generated from TSPLIB. We will only run the algorithm on the instances that are feasible to run on our machine, namely, small random instances. If you want to test other instances, just add them to the `hardtsplib_instances` list at position 0.

In [28]:
hardtsplib_instances = [("10007_hard", 10), ("10010_hard", 10), ("10008_hard", 10), ("10001_hard", 10), ("11675_hard", 11), ("12290_hard", 12), ("14850_hard", 14), ("15005_hard", 15), ("15002_hard", 15), ("15007_hard", 15), ("16038_hard", 16)]